In [94]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 500)
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import json

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer, PorterStemmer 

from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from itertools import islice
from scipy.spatial.distance import cdist
from scipy.stats.stats import pearsonr
import enchant
import os

In [95]:
import googletrans
from googletrans import Translator
import time
import re
import unicodedata

In [96]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [97]:
DATA_FOLDER = "data"
data_path = os.path.join(DATA_FOLDER) + "/"

if not DATA_FOLDER in os.listdir():
    os.mkdir(DATA_FOLDER)

# Read csv

In [98]:
df = pd.read_csv(data_path+'Beer_dataset_v2.csv', index_col=0)

In [99]:
df.head()

,Name,Style,ABV,IBU,Rating,Description,Brewery_full,Description_full,Description_full_clean,Name_clean
0,La Vella Caravana / Black Bitch Caravan of Bitches,Witbier,5.2,29.0,NaN,"Caravan of Bitches\nHoppy Wheat Ale\nEstilo tradicional elaborado con malta pilsner, trigo y avena. Este estilo se caracteriza por las adiciones de cascara de naranja amarga y semillas de cilantro.",La Vella CaravanaBlack Bitch Brewing Co,"Caravan of Bitches\nHoppy Wheat Ale\nEstilo tradicional elaborado con malta pilsner, trigo y avena. Este estilo se caracteriza por las adiciones de cascara de naranja amarga y semillas de cilantro.","estilo tradicional elaborado con malta pilsner, trigo y avena. este estilo se caracteriza por las adiciones de cascara de naranja amarga y semillas de cilantro.",la vella caravana black bitch caravan of bitches
1,BlackLab Sour Lady,Berliner Weisse,5.0,NaN,NaN,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,BlackLab,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,sour beer with raspberries,blacklab sour lady
2,Blacklab Claudia Passion,American IPA,NaN,NaN,NaN,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,BlackLab,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,our best ipa with passion fruit,blacklab claudia passion
3,BlackLab More Sparkle,Brut IPA,7.0,NaN,NaN,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,BlackLab,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,muy seca y aromática. poco amargor.,blacklab more sparkle
4,Greyhound Brewers Give ’em Hops,Imperial IPA,8.0,NaN,NaN,Give ’em Hops\nDoble NEIPA,Greyhound Brewers,Give ’em Hops\nDoble NEIPA,doble neipa,greyhound brewers give em hops


In [100]:
columns = ['Name_clean', 'Brewery_full', 'Style', 'ABV', 'IBU', 'Description_full', 'Description_full_clean']
df = df[columns]

In [101]:
df.head()

,Name_clean,Brewery_full,Style,ABV,IBU,Description_full,Description_full_clean
0,la vella caravana black bitch caravan of bitches,La Vella CaravanaBlack Bitch Brewing Co,Witbier,5.2,29.0,"Caravan of Bitches\nHoppy Wheat Ale\nEstilo tradicional elaborado con malta pilsner, trigo y avena. Este estilo se caracteriza por las adiciones de cascara de naranja amarga y semillas de cilantro.","estilo tradicional elaborado con malta pilsner, trigo y avena. este estilo se caracteriza por las adiciones de cascara de naranja amarga y semillas de cilantro."
1,blacklab sour lady,BlackLab,Berliner Weisse,5.0,NaN,Sour Lady\nBerliner Weisse with fruit\nUna cerveza ácida con frambuesas.\n---------------\nSour beer with raspberries,sour beer with raspberries
2,blacklab claudia passion,BlackLab,American IPA,NaN,NaN,Claudia Passion\nFruit IPA\nNuestra mejor cerveza con Maracuyá\n--------------\nOur best IPA with passion fruit,our best ipa with passion fruit
3,blacklab more sparkle,BlackLab,Brut IPA,7.0,NaN,More Sparkle\nBrut IPA\nDry and aromatic. Low bitterness.\n-----------\nMuy seca y aromática. Poco amargor.,muy seca y aromática. poco amargor.
4,greyhound brewers give em hops,Greyhound Brewers,Imperial IPA,8.0,NaN,Give ’em Hops\nDoble NEIPA,doble neipa


In [102]:
df.isna().sum()

Name_clean                   0
Brewery_full               385
Style                        0
ABV                        185
IBU                       3298
Description_full           413
Description_full_clean     413
dtype: int64

In [103]:
df.shape

(7879, 7)

In [104]:
df[df.Description_full.isna()]

,Name_clean,Brewery_full,Style,ABV,IBU,Description_full,Description_full_clean
29,el pedal export stout,NaN,Foreign Extra Stout,6.0,NaN,NaN,NaN
33,garage beer co scripted spiel wipa,NaN,American IPA,5.5,NaN,NaN,NaN
64,a cova céltica fartura,NaN,American Pale Ale,5.5,NaN,NaN,NaN
103,click brew iler,NaN,American Pale Ale,0.0,NaN,NaN,NaN
105,albert the great batch 1,NaN,Stout,12.0,40.0,NaN,NaN
159,beering barcelona fight ink,NaN,American IPA,6.0,60.0,NaN,NaN
207,cervezahara la cachonda,NaN,India Pale Ale,6.8,50.0,NaN,NaN
252,ruben s schwarz 2,NaN,Schwarzbier (Black Beer),4.3,NaN,NaN,NaN
321,kusfollin en el nom del porc,NaN,New England IPA,4.7,NaN,NaN,NaN
333,entre fronteras la chisparrera,NaN,Belgian Blonde Ale,6.0,NaN,NaN,NaN


# Stopwords

In [105]:
df['Description_full_clean'] = df['Description_full_clean'].fillna('')

In [106]:
stopwords_list = stopwords.words('english') + stopwords.words('spanish')
df['Description_2'] = df['Description_full_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

# Unique words

In [107]:
df["Description_2"] = df["Description_2"].apply(lambda x: pd.unique(x.split(' ')))
df["Description_2"] = df["Description_2"].apply(lambda x: ' '.join(x))

In [108]:
df[['Name_clean', 'Description_full', 'Description_2']].sample(5)

,Name_clean,Description_full,Description_2
4149,sra maltworth,NaN,
4262,dobleerre imperial rufis ii,NaN,
1477,península hazy vibes columbus amarillo,"Hazy Vibes: Columbus y Amarillo\nDDH IPA de 40 IBUS, forma parte de nuestra nueva Hazy Vibes Series. Esta IPA con Double Dry-Hop de Columbus y Amarillo es seca y refrescante a pesar de sus 7 grados. El aroma recuerda al de la naranja y el pomelo con ... Ver más leve especiado aportado por el Columbus. Ver menos","ddh ipa 40 ibus, forma parte nueva hazy vibes series. double dry-hop columbus amarillo seca refrescante pesar 7 grados. aroma recuerda naranja pomelo ... ver leve especiado aportado columbus. menos"
32,edge brewing frequency,Frequency\nHoppy Golden Ale,hoppy golden ale
3184,toupiña une,"Toupiña UNE\r\nProfundo vermello-cobrizo, moderadamente forte, maltosa, complexa, con ricos sabores de malte, ésteres a froitas escuras ou secas, e lixeiro alcol mesturados xuntos nunha presentación que aunque maltosa remata bastante seca.","profundo vermello-cobrizo, moderadamente forte, maltosa, complexa, ricos sabores malte, ésteres froitas escuras ou secas, lixeiro alcol mesturados xuntos nunha presentación aunque maltosa remata bastante seca."


# Remove words

In [109]:
list_words_to_drop = ['beer', 'beers', 'cerveza', 'cervezas', 'la', 'una', 'elaborada', 'el', 'hop', 'hops', 'lupulo', 'lupulos']

In [110]:
df["Description_2"] = df["Description_2"].apply(lambda x: [word for word in x.split() if word not in list_words_to_drop])
df["Description_2"] = df["Description_2"].apply(lambda x: ' '.join(x))
df['Description_2'] = df['Description_2'].map(lambda x: re.sub(r'\W+', ' ', x)).str.lower()

In [111]:
df[['Description_full', 'Description_2']].sample(5)

,Description_full,Description_2
1132,Sisyphus\nGalaxy DDH IPA,galaxy ddh ipa
1962,NaN,
2742,"Estrella Galicia Pimientos Padrón\r\nUnas pican e outras non.\r\nCerveza de pimientos de Padrón, con lúpulo gallego, tiene 6,5 grados de alcohol y un picante que difiere en función de la apreciación de cada consumidor.",pimientos padrón lúpulo gallego 6 5 grados alcohol picante difiere función apreciación cada consumidor
7657,"Bayura Mexu de Trasgu India Pale Ale doble carga de lúpulo\r\nAromática con marcados matices cítricos y tropicales, aparecen tambien toques de resina de pino y de bosque. En boca es amarga, con recuerdos resinosos, cítricos y ligeramente minerales. ... Ver más seco y bien amargo. Ver menos",bayura aromática marcados matices cítricos tropicales aparecen tambien toques resina pino bosque boca amarga recuerdos resinosos ligeramente minerales ver seco bien amargo menos
1449,"Gayaspera\nFolixa asturiana en una botella. Las maltas tostadas y los lúpulos nobles alemanes dan carácter a esta lager de tonos rojizos, cargada de aromas y sabores a caramelo y un ligero final torrefacto.",folixa asturiana botella maltas tostadas lúpulos nobles alemanes dan carácter lager tonos rojizos cargada aromas sabores caramelo ligero final torrefacto


# English dictionary

In [112]:
df['Description_3'] = df['Description_2'].apply(lambda x: pd.unique(x.split()))

In [113]:
results = set()
df['Description_3'].apply(results.update)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
7849    None
7850    None
7851    None
7852    None
7853    None
7854    None
7855    None
7856    None
7857    None
7858    None
7859    None
7860    None
7861    None
7862    None
7863    None
7864    None
7865    None
7866    None
7867    None
7868    None
7869    None
7870    None
7871    None
7872    None
7873    None
7874    None
7875    None
7876    None
7877    None
7878    None
Name: Description_3, Length: 7879, dtype: object

In [114]:
results = list(results)  

In [115]:
len(results)  # number of unique words

25790

In [116]:
d = enchant.Dict("en_US")
# d.check("Hello")

In [117]:
results_english = []

for element in results:
    if d.check(element):
        results_english.append(element)

In [118]:
len(results_english)  # number of unique english words

3611

# Translator

In [119]:
# dict_trans = {}

In [120]:
# translator = Translator()

In [121]:
# for element in results_english:
#     if element not in dict_trans.keys():
#         dict_trans[element] = translator.translate(element, src="en", dest="es").text

In [122]:
# with open('dict_english.json', 'w') as fp:
#     json.dump(dict_trans, fp)

In [123]:
with open('dict_english.json') as json_file:
    dict_trans = json.load(json_file)

In [124]:
len(dict_trans)

3755

In [125]:
english_words = ['pale', 'ale', 'ales', 'brew', 'shandy', 'shandys', 'hazy', 'dry', 'warrior',
                 'galaxy', 'mosaic', 'england', 'new', 'porter', 'toque', 'hops', 'cashmere', 'melon', 'secret', 'centennial']

In [126]:
def func_lambda(x):
    text = []    
    for element in x:
        if element in dict_trans.keys() and element not in english_words:
            element = dict_trans[element]
        text.append(element)
    return ' '.join(text)

In [127]:
df['Description_spanish'] = df['Description_3'].apply(func_lambda).str.lower()
df["Description_spanish"] = df["Description_spanish"].apply(lambda x: pd.unique(x.split(' ')))
df["Description_spanish"] = df["Description_spanish"].apply(lambda x: ' '.join(x))

In [128]:
# df["Description_spanish"] = df["Description_spanish"].apply(lambda x: [int(s) for s in x.split() if not s.isdigit()])
# df["Description_spanish"] = df["Description_spanish"].apply(lambda x: ' '.join(x))
# re.findall(r'\d+', 'hello 42 I\'m a 32 string 30')

In [129]:
df[['Name_clean', 'Description_full', 'Description_spanish']].sample(5)

,Name_clean,Description_full,Description_spanish
5729,the angry brewery beleak,The angry Brewery Beleak\n\r\nExtra Stout,extra cerveza negra
4659,toutatis la blonde,"Toutatis La Blonde\r\nCerveza rubia de estilo Belga que a diferencia de las rubias pale ale y lager típicas, tiene un dulzor y cuerpo más marcado, combinado con frutados procedentes de la levadura.\r\nLa sensación que aporta la malta de estilo Belga ... Ver más los frutados procedentes de la levadura de abadías hacen de ésta una cerveza compleja sin necesidad de ingredientes aromatizantes ni lúpulos marcados. Ver menos",rubia estilo belga diferencia rubias pale ale lager típicas dulzor cuerpo marcado c sensación aporta malta ver frutados procedentes levadura abadías hacen ésta compleja necesidad ingredientes aromatizantes lúpulos marcados menos
1311,barcelona beer company santa rita,Santa Rita\nPremium Craft Lager,prima arte lager
93,basqueland oloroso beltza,Oloroso Beltza\nImperial Stout aged in Oloroso barrels,imperial cerveza negra envejecido oloroso barriles
5333,la quince horny pilsner,La Quince Horny Pilsner\r\nUna Hoppy Pils para beber sin parar,hoppy pils beber parar


# Extract hops

In [130]:
hops_list = ['mosaic', 'citra', 'amarillo', 'simcoe', 'cascade', 'columbus',
             'chinook', 'cashmere', 't90', 'ekuanot', 'mandarina bavaria', 'galaxy', 'idaho 7',
             'hüll melon', 'enigma', 'vic secret', 'willamette', 'warrior', 'centennial', 'nelson sauvin', 'galena', 'ahtanum', 'mandarina']

In [131]:
# TO DO
# Incluir lúpulos de dos palabras, ahora mismo se están sacando los de una única palabra

In [132]:
df['hops'] = df['Description_spanish'].apply(lambda x: ([word for word in x.split() if word in (hops_list)]))

In [133]:
df.sample(2)

,Name_clean,Brewery_full,Style,ABV,IBU,Description_full,Description_full_clean,Description_2,Description_3,Description_spanish,hops
3442,filabres american pale ale,Cervezas de los Filabres,American Pale Ale,5.0,NaN,Filabres American Pale Ale\r\nElaborada en Velefique con dos tipos de maltas base y dos especiales además de los lúpulos en flor. Aroma y sabor dominado por el carácter de los cereales. Cerveza de fermentación alta y ligeramente especiad,elaborada en velefique con dos tipos de maltas base y dos especiales además de los lúpulos en flor. aroma y sabor dominado por el carácter de los cereales. cerveza de fermentación alta y ligeramente especiad,velefique dos tipos maltas base especiales además lúpulos flor aroma sabor dominado carácter cereales fermentación alta ligeramente especiad,"[velefique, dos, tipos, maltas, base, especiales, además, lúpulos, flor, aroma, sabor, dominado, carácter, cereales, fermentación, alta, ligeramente, especiad]",velefique dos tipos maltas base especiales además lúpulos flor aroma sabor dominado carácter cereales fermentación alta ligeramente especiad,[]
5952,hopland a n a ai pi ei,Hopland Cerveceros,English IPA,4.9,62.0,A.N.A. Ai Pi Ei es la evolución de nuestra Blonde Ale A.N.A. A Naughty Ale. Pertenece a la categoria de las English IPA. Su amargor es moderado y la gran variedad de lúpulos empleados le dan un gran y complejo aroma. Su color es ambar.,a.n.a. ai pi ei es la evolución de nuestra blonde ale a.n.a. a naughty ale. pertenece a la categoria de las english ipa. su amargor es moderado y la gran variedad de lúpulos empleados le dan un gran y complejo aroma. su color es ambar.,a n a ai pi ei evolución blonde ale naughty ale pertenece categoria english ipa amargor moderado gran variedad lúpulos empleados dan complejo aroma color ambar,"[a, n, ai, pi, ei, evolución, blonde, ale, naughty, pertenece, categoria, english, ipa, amargor, moderado, gran, variedad, lúpulos, empleados, dan, complejo, aroma, color, ambar]",a norte ai pi ei evolución rubia ale travieso pertenece categoria english ipa amargor moderado gran variedad lúpulos empleados dan complejo aroma color ambar,[]


# Extract fruits

In [134]:
fruit_list = ['naranja', 'pomelo', 'mango', 'frambuesas']

In [135]:
df['fruits'] = df['Description_spanish'].apply(lambda x: ([word for word in x.split() if word in (fruit_list)]))

In [136]:
df.sample(2)

,Name_clean,Brewery_full,Style,ABV,IBU,Description_full,Description_full_clean,Description_2,Description_3,Description_spanish,hops,fruits
4901,tu aperitivo compartir,Tu AperitivoBirra 08,Pale Ale,5.4,NaN,"Hemos creado junto con los Cerveceros Artesanales de Birra08, la Cerveza Artesanal Compartir, la Cerveza del Aperitivo. Una Cerveza del estilo Pale Ale con 5,4% de alcohol. Una Cerveza diseñada para disfrutar de los complementos de un buen Aperitivo. ... Ver más Cerveza de tuaperitivo.\r\nEn nariz presenta toques cítricos de mandarina u orangina. Con ligero aroma floral y con toque de vainilla ligera.\r\nEn boca es equilibrada, con un punto de aguja que acentua su carácter fresco. Encontramo...","hemos creado junto con los cerveceros artesanales de birra08, la cerveza artesanal compartir, la cerveza del aperitivo. una cerveza del estilo pale ale con 5,4% de alcohol. una cerveza diseñada para disfrutar de los complementos de un bu en nariz presenta toques cítricos de mandarina u orangina. con li en boca es equilibrada, con un punto de aguja que acentua su carácter fresco. encontramos las maltas integradas y un final cítrico y seco. este final seco, la hace ideal para el ap para tomar ...",creado junto cerveceros artesanales birra08 artesanal compartir aperitivo estilo pale ale 5 4 alcohol diseñada disfrutar complementos bu nariz presenta toques cítricos mandarina u orangina li boca equilibrada punto aguja acentua carácter fresco encontramos maltas integradas final cítrico seco seco hace ideal ap tomar conservas patatas fritas gourmet aceitunas ver menos,"[creado, junto, cerveceros, artesanales, birra08, artesanal, compartir, aperitivo, estilo, pale, ale, 5, 4, alcohol, diseñada, disfrutar, complementos, bu, nariz, presenta, toques, cítricos, mandarina, u, orangina, li, boca, equilibrada, punto, aguja, acentua, carácter, fresco, encontramos, maltas, integradas, final, cítrico, seco, hace, ideal, ap, tomar, conservas, patatas, fritas, gourmet, aceitunas, ver, menos]",creado junto cerveceros artesanales birra08 artesanal compartir aperitivo estilo pale ale 5 4 alcohol diseñada disfrutar complementos bu nariz presenta toques cítricos mandarina u orangina li boca equilibrada punto aguja acentua carácter fresco encontramos maltas integradas final cítrico seco hace ideal ap tomar conservas patatas fritas gastrónomo aceitunas ver menos,[mandarina],[]
5635,crápula,Cerveza Crápula,American Pale Ale,5.1,38.0,"Crápula\n\r\nCrápula es un individuo al que le gusta el desenfreno, no tiene la vida ordenada, es inmoral y esta en contra de las buenas costumbres.\n\r\nAmerican Pale Ale con 3 maltas y 4 lúpulos americanos.","crápula es un individuo al que le gusta el desenfreno, no tiene la vida ordenada, es american pale ale con 3 maltas y 4 lúpulos americanos.",crápula individuo gusta desenfreno vida ordenada american pale ale 3 maltas 4 lúpulos americanos,"[crápula, individuo, gusta, desenfreno, vida, ordenada, american, pale, ale, 3, maltas, 4, lúpulos, americanos]",crápula individuo gusta desenfreno vida ordenada american pale ale 3 maltas 4 lúpulos americanos,[],[]


# NLP

## Stemming

In [137]:
# Keep root of words

stemmer = SnowballStemmer("spanish")
df['Spanish_stemmed'] = df["Description_spanish"].map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
# df["stemmed_clean"] = df['stemmed'].map(lambda x: re.sub(r'\W+', ' ', x))

In [138]:
df[['Name_clean', 'Description_spanish', 'Spanish_stemmed']].sample(5)

,Name_clean,Description_spanish,Spanish_stemmed
7357,archi pilsen bohemia premium,natural inconfundible color dorado sabor auténtico duradero fruto mejor tradición cervecera exclusivamente agua madrid 100 pura malta cebada selección lúpulos nobles levadura,natural inconfund color dor sabor autent durader frut mejor tradicion cervecer exclus agu madr 100 pur malt ceb seleccion lupul nobl levadur
4873,mangurria yihipa,artesana ecológica,artesan ecolog
5550,gara tomasa,gara tomasa negra dunkel bock,gar tomas negr dunkel bock
3919,fort barcelona pils,fuerte bcn pils rene cada trago autenticidad bien hecha,fuert bcn pils ren cad trag autent bien hech
7181,la baturra tostada,artesanal intensa delicada saborear forma pausada relajada maltas alta calidad mezcla lúpulos levaduras tipo ale color tostado sabor caramelo aromas recuerdan cereales tostados notas ver secos ideal acompañar carnes menos,artesanal intens delic sabor form paus relaj malt alta calid mezcl lupul levadur tip ale color tost sabor caramel arom recuerd cereal tost not ver sec ideal acompañ carn men


## PorterStemmer

In [139]:
# stemmer = PorterStemmer()
# df['Description_3'] = df["Description_3"].map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))

## CountVectorizer

In [140]:
cvec = CountVectorizer(
#     stop_words=["english", "spanish"], 
    stop_words=["spanish"],     
    min_df=0.01, 
    max_df=0.50, 
    ngram_range=(1,3) # change to consider bigrams
#     ngram_range=(1) # unigrams
)
cvec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=0.01,
                ngram_range=(1, 3), preprocessor=None, stop_words=['spanish'],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [141]:
# cvec.fit(df['Description_3'])
# cvec.fit(df['Description_spanish'])
cvec.fit(df['Spanish_stemmed'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=0.01,
                ngram_range=(1, 3), preprocessor=None, stop_words=['spanish'],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [142]:
cvec.get_feature_names()

['10',
 '100',
 'abund',
 'abv',
 'acab',
 'acid',
 'acidez',
 'acompañ',
 'ademas',
 'adicion',
 'afabl',
 'afrut',
 'agrad',
 'agri',
 'agu',
 'ahum',
 'alc',
 'alcohol',
 'ale',
 'ale color',
 'aleman',
 'alta',
 'alta ferment',
 'alto',
 'amarg',
 'amargor',
 'amargur',
 'amarill',
 'amb',
 'ambar',
 'americ',
 'americ pal',
 'americ pal ale',
 'american',
 'amig',
 'anaranj',
 'aparient',
 'aperit',
 'aport',
 'apreci',
 'aquest',
 'arom',
 'arom citric',
 'arom frut',
 'arom intens',
 'arom lupul',
 'arom malt',
 'arom sabor',
 'aromat',
 'artesan',
 'artesanal',
 'asad',
 'asi',
 'aspect',
 'aunqu',
 'autent',
 'aven',
 'azuc',
 'azul',
 'añad',
 'año',
 'años',
 'baj',
 'balanc',
 'barric',
 'barril',
 'bas',
 'beb',
 'belg',
 'belgi',
 'bien',
 'blanc',
 'blat',
 'boc',
 'botell',
 'brillant',
 'bronc',
 'buen',
 'burbuj',
 'busc',
 'caca',
 'cad',
 'caf',
 'calid',
 'cantid',
 'caract',
 'caracterist',
 'caracteriz',
 'caramel',
 'carameliz',
 'carbon',
 'carbonat',
 'carbona

In [143]:
list(islice(cvec.vocabulary_.items(), 20))

[('estil', 200),
 ('tradicional', 444),
 ('elabor', 179),
 ('malt', 279),
 ('pilsn', 351),
 ('trig', 448),
 ('aven', 56),
 ('caracteriz', 87),
 ('adicion', 9),
 ('casc', 97),
 ('naranj', 314),
 ('amarg', 24),
 ('cilantr', 111),
 ('agri', 13),
 ('mejor', 299),
 ('ipa', 255),
 ('frut', 217),
 ('sec', 407),
 ('aromat', 48),
 ('amargor', 25)]

In [144]:
# cvec_counts = cvec.transform(df['Description_3'])
# cvec_counts = cvec.transform(df['Description_spanish'])
cvec_counts = cvec.transform(df['Spanish_stemmed'])

print('sparse matrix shape:', cvec_counts.shape)
print('nonzero count:', cvec_counts.nnz)
print('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

sparse matrix shape: (7879, 469)
nonzero count: 124299
sparsity: 3.36%


In [145]:
# top 20 terms
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})

In [146]:
counts_df.sort_values(by='occurrences', ascending=False).head(10)

,term,occurrences
301,men,3721
460,ver,3720
279,malt,2613
41,arom,2545
273,lupul,2489
399,sabor,2235
119,color,1944
264,liger,1496
18,ale,1496
25,amargor,1250


## TfidfTransformer

In [147]:
# use tfidf transformer to calc weights
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<7879x469 sparse matrix of type '<class 'numpy.float64'>'
	with 124299 stored elements in Compressed Sparse Row format>

In [148]:
# beer_name = 'Malandar Silk'
# beer_name = 'Arriaca Triple IPA'
# beer_name = 'Alegría del Barrio'
beer_name = 'la grúa bamboo'

# Compute cosine similarity with a beer of choice, that is represented in the dataset
idx = df.index.get_indexer_for((df[df['Name_clean']==beer_name].index))[0]

In [149]:
df[df.Name_clean.str.contains('alegría')]['Name_clean']

987         alegría pink revolution ipa 2019
1889    grx brewing alegría comandante mango
3030                alegría del mediterráneo
4322      alegría boris brew ipanosuarus rex
4665     alegría pirate brew juanito polarez
4787                        alegría de vivir
5045                     la jerezana alegría
6986                       alegría del valle
6987                         alegría del sur
6988                      alegría del barrio
6989                          alegría mcmiau
Name: Name_clean, dtype: object

## Cosine Similarity

In [150]:
df_recommendation = df[['Name_clean', 'Brewery_full', 'IBU', 'Style', 'Description_full', 'Description_spanish', 'hops']]

In [151]:
sims = cosine_similarity(transformed_weights[idx, :], transformed_weights)

# Add similarities values to the beer description dataframe, then sort by similarities descending
df_recommendation['sims'] = sims.T
df_recommendation.sort_values(by='sims', ascending=False).head(20)

/home/cx02358/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Name_clean,Brewery_full,IBU,Style,Description_full,Description_spanish,hops,sims
405,la grúa bamboo,Cervezas La Grúa,NaN,New England IPA,"Bamboo\nDDH NEIPA\nCon lúpulos Idaho 7, Mosaic, Cashmere y avena\n",lúpulos idaho 7 mosaic cashmere avena,"[mosaic, cashmere]",1.000000
3531,mad brewing camarillo,Mad Brewing,NaN,India Pale Ale,Mad Brewing Camarillo\r\nIPA con avena y lúpulo Mosaic\n,ipa avena lúpulo mosaic,[mosaic],0.912488
3448,la quince 15 hours session ipa mosaic,La Quince Brewery,55.0,India Pale Ale,La Quince 15 Hours Session IPA Mosaic\r\nMonovarietal de lúpulo Mosaic.,monovarietal lúpulo mosaic,[mosaic],0.753122
1241,biribil eguzkilore,Biribil Brewing,NaN,New England IPA,"Eguzkilore\nHazy IPA\nAvena, turbiedad, levadura Vermont y lúpulos Mosaic y Amarillo a paladas para un aroma espectacular.",avena turbiedad levadura vermont lúpulos mosaic amarillo paladas aroma espectacular,"[mosaic, amarillo]",0.716840
1322,the flying inn lilith,The Flying Inn,70.0,India Pale Ale,"Lilith\nIPA Pale malt, oat malt, wheat malt, cara malt. Mosaic, Ekuanot.",ipa pale malta avena trigo cara mosaic ekuanot,"[mosaic, ekuanot]",0.691430
1504,baias singapore,Baias Garagardotegia,NaN,India Pale Lager,Singapore\nIPL 6%\nMosaic,mosaic,[mosaic],0.678311
1670,basqueland brut reynolds,Basqueland Brewing Project,NaN,India Pale Ale,"Brut Reynolds\nDDH Brut IPA\nA beer dedicated to the late, great Burt Reynolds.\nCryo Simcoe\nCryo Mosaic",cryo mosaic,[mosaic],0.678311
6131,tro ales single hop mosaic,Tro Ales,50.0,American Pale Ale,Tro Ales Single Hop Mosaic\n,tro ales soltero mosaic,[mosaic],0.678311
1919,laugar brussels beer project the lauter revenge,Laugar BreweryBrussels Beer Project,NaN,American IPA,The Lauter Revenge\r\nNEIPA con avena,neipa avena,[],0.657880
1091,la quince señor lúpulo tsunami neipa,La Quince BrewerySeñor Lúpulo,NaN,New England IPA,"Tsunami NEIPA\nCollab with Señor Lúpulo Gijón\nMalts: golden promise, wheat and oats. Hops: Ekuanot, Mosaic, Summit, Polaris and Cascade. Vermont yeast",maltas dorado promesa trigo avena hops ekuanot mosaic cumbre polaris cascada vermont levadura,"[ekuanot, mosaic]",0.597808
